In [2]:
# !pip install -r /content/requirements.txt

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores.faiss import FAISS
from langchain.embeddings import OpenAIEmbeddings
import pickle
import os

In [1]:
os.environ["OPENAI_API_KEY"]=""


NameError: name 'os' is not defined

In [12]:
path="/content/drive/MyDrive/XML_data"
document=[]
for file in os.listdir(path):
  if file.endswith(".pdf"):
    pdf_path="/content/drive/MyDrive/XML_data/"+file
    print(pdf_path)
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    text_path="/content/drive/MyDrive/XML_data/"+file
    print(text_path)
    loader=TextLoader(text_path)
    document.extend(loader.load())

/content/drive/MyDrive/XML_data/state_of_the_union.txt
/content/drive/MyDrive/XML_data/langchain.pdf


In [13]:
document_splitter=CharacterTextSplitter(separator='\n',
                                        chunk_size=500,
                                        chunk_overlap=100)

In [14]:
document_chunks=document_splitter.split_documents(document)


In [15]:
DB_FAISS_PATH = "vectorstore"
print("Creating vectorstore...")
embeddings = OpenAIEmbeddings()
print("sommmmmmmmmmmmm")
vectorstore = FAISS.from_documents(document_chunks, embeddings)

vectorstore.save_local(DB_FAISS_PATH)
print("done")

Creating vectorstore...
sommmmmmmmmmmmm
done


In [16]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
import pickle
import sys

In [17]:
def load_retriever():
    retriever = FAISS.load_local("/content/vectorstore", embeddings)
    return retriever
def get_basic_qa_chain():
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    retriever = load_retriever()
    memory = ConversationBufferMemory(
        memory_key="chat_history", return_messages=True)
    model = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever.as_retriever(),verbose=False,
        memory=memory)
    return model

In [18]:
chain=get_basic_qa_chain()
output = chain({"question":"What did the president say about Ketanji Brown Jackson"})
print(output['answer'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The President mentioned that Ketanji Brown Jackson is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He highlighted her background as a former top litigator in private practice, a former federal public defender, and coming from a family of public school educators and police officers. He also mentioned that since her nomination, she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans.


In [25]:
chain=get_basic_qa_chain()
output = chain({"question":"Give me two interesting point about MARCIUS"})
print(output['answer'])

1. Marcius is portrayed as a fierce and determined warrior, willing to fight even when injured, as seen when he insists on continuing to fight despite bleeding and exhaustion. This showcases his unwavering dedication to his cause and his resilience in the face of physical challenges.

2. Marcius is shown to have a complex character, as he struggles with feelings of envy towards his rival, Tullus Aufidius, despite recognizing his noble qualities. This internal conflict adds depth to his character and highlights the internal struggles he faces while navigating the political and military landscape.


In [26]:
chain=get_basic_qa_chain()
output = chain({"question":"What was his stance on Ukraine?"})
print(output['answer'])

The speaker in the provided context strongly supports Ukraine and its people in their fight against Russian aggression. The speaker emphasizes the courage, determination, and fearlessness of the Ukrainian people and calls for solidarity with Ukraine in the face of the ongoing conflict.


In [28]:
chain=get_basic_qa_chain()
output = chain({"question":"who is First Senator? and what is the role "})
print(output['answer'])

The First Senator is a character in the play or text being referenced. The role of the First Senator is likely to be a member of the Senate or a political figure involved in the governance of the state or city mentioned in the text.
